In [10]:
import pandas as pd
import sys
import csv
from statistics import median

In [11]:
project_ytest_lib = {}

In [12]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [13]:
def get_best_confidence(data):
    
    #typecasting
    data['builds_reqd'] = pd.to_numeric(data['builds_reqd'], downcast="float", errors='coerce')   
    
    #setting parameters
    project_list = set(data['project'].tolist())
    batching_alg = set(data['algorithm'].tolist())
    batchsizes = [1,2,4,8,16]
    
    result_rows = []
    
    #get best confidences

    for p in project_list:
        
        p_name = p + '.csv'
        alg_scores = {}
        p_data = data[data['project'] == p]
        
        
        for alg in batching_alg:

            alg_data = p_data[p_data['algorithm'] == alg]

            for b in batchsizes:

                if alg == 'BATCH4':
                    if b != 4:
                        continue

                if alg == 'BATCHSTOP4':
                    if b < 4:
                        continue
                        

                batch_data = alg_data[ alg_data['batch_size'] == b]
                
                total = batch_data['testall_size'].tolist()[0]
                reqd_list = batch_data['builds_reqd'].tolist()
                median_delays = batch_data['median_delay'].tolist()
                delay_lists = batch_data['delay_list'].tolist()
                actual_durations = batch_data['actual_duration'].tolist()
                build_durations = batch_data['build_duration'].tolist()
                
                min_distance = sys.maxsize

                best = [p_name, 'tr', alg, b, 0, 0, [], 0, 0, 0, 0]
                best_i = 0

                for i in range(len(reqd_list)):
                    distance = ((reqd_list[i]**2)+(median_delays[i])**2)**0.5
                    if distance < min_distance:
                        best[4] = reqd_list[i]
                        best[5] = median_delays[i]
                        best[6] = delay_lists[i]
                        best[7] = total
                        best[8] = i+2
                        best[9] = actual_durations[i]
                        best[10] = build_durations[i]
                        best_i = i
                        min_distance = distance

                result_rows.append(best)
    
    return result_rows

In [14]:
project_list = ['heroku', 'open-build-service', 'cloud_controller_ng', 'rubinius', 'jruby', 'metasploit-framework', 'loomio', 'orbeon-forms', 'cloudify', 'geoserver', 'opal', 'sufia', 'sonarqube', 'rails', 'fog', 'puppet', 'concerto', 'vagrant', 'gradle', 'graylog2-server']

In [15]:
data = pd.read_csv('final_tr_results.csv')
result_rows = get_best_confidence(data)

In [16]:
len(result_rows)

180

In [17]:
df = pd.DataFrame(result_rows, columns=['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'median_delay', 'delay_list', 'testall_size', 'confidence', 'actual_durations', 'build_durations'])

In [18]:
df.to_csv('best_confidence.csv')